In [1]:
%matplotlib inline
import rasterio
import matplotlib.pyplot as plt
import gdal
import numpy as np
import timeit

In [2]:
# Function list
def band_average(raster, bands, icethickness):
    band_average = np.zeros((bands.max()+1, 1))
    for n in range(bands.max()+1):
        raster_single = raster.copy()
        raster_single[bands != n] = np.nan
        raster_single[(icethickness == 0) & (emergence_velocity == 0)] = np.nan
        band_average[n,0] = np.nanmean(raster_single)
    return band_average
    

def emergence_pixels(vel_x_raw, vel_y_raw, vel_total, vel_min, vel_depth_avg_factor, icethickness, bands):
    """ Compute the emergence velocity using an ice flux approach
    """
    # Modify vel_y by multiplying velocity by -1 such that matrix operations agree with flow direction
    #    Specifically, a negative y velocity means the pixel is flowing south.
    #    However, if you were to subtract that value from the rows, it would head north in the matrix.
    #    This is due to the fact that the number of rows start at 0 at the top.
    #    Therefore, multipylying by -1 aligns the matrix operations with the flow direction
    vel_y = vel_y_raw * -1 * vel_depth_avg_factor
    vel_x = vel_x_raw * vel_depth_avg_factor
    # Compute the initial volume
    volume_initial = icethickness * pixel_size**2
    # Quality control options:
    # Apply a border based on the max specified velocity to prevent errors associated with pixels going out of bounds
    if option_border == 1:
        border = int(max_velocity / pixel_size)
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if (r < border) | (r >= vel_x.shape[0] - border) | (c < border) | (c >= vel_x.shape[1] - border):
                    vel_x[r,c] = 0
                    vel_y[r,c] = 0
    # Minimum/maximum velocity bounds
    if option_minvelocity == 1:
        vel_x[vel_total < vel_min] = 0
        vel_y[vel_total < vel_min] = 0
    if option_maxvelocity == 1:
        vel_x[vel_total > max_velocity] = 0
        vel_y[vel_total > max_velocity] = 0
    # Remove clusters of high velocity on stagnant portions of glaciers due to feature tracking of ice cliffs and ponds
    if option_stagnantbands == 1:
        vel_x[bands <= stagnant_band] = 0
        vel_y[bands <= stagnant_band] = 0        
    # Compute displacement in units of pixels
    vel_x_pix = vel_x / pixel_size
    vel_y_pix = vel_y / pixel_size
    # Compute the displacement and fraction of pixels moved for all columns (x-axis)
    # col_x1 is the number of columns to the closest pixel receiving ice [ex. 2.6 returns 2, -2.6 returns -2]
    #    int() automatically rounds towards zero
    col_x1 = vel_x_pix.astype(int)
    # col_x2 is the number of columns to the further pixel receiving ice [ex. 2.6 returns 3, -2.6 returns -3]
    #    np.sign() returns a value of 1 or -1, so it's adding 1 pixel away from zero
    col_x2 = (vel_x_pix + np.sign(vel_x_pix)).astype(int)
    # rem_x2 is the fraction of the pixel that remains in the further pixel (col_x2) [ex. 2.6 returns 0.6, -2.6 returns 0.6]
    #    np.sign() returns a value of 1 or -1, so multiplying by that ensures you have a positive value
    #    then when you take the remainder using "% 1", you obtain the desired fraction
    rem_x2 = np.multiply(np.sign(vel_x_pix), vel_x_pix) % 1
    # rem_x1 is the fraction of the pixel that remains in the closer pixel (col_x1) [ex. 2.6 returns 0.4, -2.6 returns 0.4]
    rem_x1 = 1 - rem_x2
    # Repeat the displacement and fraction computations for all rows (y-axis)
    row_y1 = vel_y_pix.astype(int)
    row_y2 = (vel_y_pix + np.sign(vel_y_pix)).astype(int)
    rem_y2 = np.multiply(np.sign(vel_y_pix), vel_y_pix) % 1
    rem_y1 = 1 - rem_y2
    # Compute the mass flux for each pixel
    volume_final = np.zeros(volume_initial.shape)
    for r in range(vel_x.shape[0]):
        for c in range(vel_x.shape[1]):
            volume_final[r+row_y1[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x1[r,c]] + rem_y1[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x1[r,c]] + rem_y2[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y1[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x2[r,c]] + rem_y1[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x2[r,c]] + rem_y2[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
    # Check that mass is conserved (threshold = 0.1 m x pixel_size**2)
    print('Mass is conserved?', (volume_final.sum() - volume_initial.sum()) < 0.1 * pixel_size**2)
    print(volume_final.sum() - volume_initial.sum())
    # Final ice thickness
    icethickness_final = volume_final / pixel_size**2
    # Emergence velocity
    emergence_velocity = icethickness_final - icethickness
    return emergence_velocity


def export_raster(raster, output_filename, ds, NoData_value):
    driver = gdal.GetDriverByName('GTiff')
    new_dataset = driver.Create(output_filename,
                                ds.RasterXSize,    # number of columns
                                ds.RasterYSize,    # number of rows
                                1,                  # number of bands
                                gdal.GDT_Float32)  # datatype of the raster
    new_dataset.SetProjection(ds.GetProjection())
    new_dataset.SetGeoTransform(ds.GetGeoTransform())
    # Now we need to set the band's nodata value to -1
    new_band = new_dataset.GetRasterBand(1)
    new_band.SetNoDataValue(NoData_value)
    # And finally, let's write our NDVI array.
    new_band.WriteArray(raster)
    

def import_raster(raster_fn):
    """Open raster and obtain the values in its first band as an array
    Output: array of raster values
    """
    # open raster dataset
    raster_ds = gdal.Open(raster_fn)
    # extract band information and get values
    raster_band = raster_ds.GetRasterBand(1)
    raster_values = raster_band.ReadAsArray()
    return raster_values

In [3]:
# # ----- Input for Khumbu Glacier ----------
# # Filenames (fn)
# vel_x_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_20132016_vx_med_ClipV2.tif'
# vel_y_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_20132016_vy_med_ClipV2.tif'
# vel_x_std_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_dshean_vx_std_ClipV2.tif'
# vel_y_std_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_dshean_vy_std_ClipV2.tif'
# icethickness_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_icethickness_Huss.tif'
# bands_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_600m_bands_polygons_raster_ClipV2.tif'
# # Additional input
# mc_simulations = 1000
# pixel_size = 30
# min_velocity_mean = 4.267
# min_velocity_std = 2.799 
# max_velocity = 200
# icethickness_uncertainty_percentage = -21
# #  Farinotti et al. [2017] showed on average Huss and Farinotti [2012] underestimate ice thickness by 21%
# vel_depth_avg_factor_min = 0.8
# vel_depth_avg_factor_max = 0.8
# #  Azam et al. (2012) uses 0.9 (state ice flux is +/- 10% based on unknown sliding and other sources of error)
# #  Nuimura et al. (2011) uses 0.8 based on Paterson (1994) and Sakai et al. (2006)
# #  Vincent et al. (2016) uses 0.8
# stagnant_band = 7

# ----- Input for Ngozumpa Glacier ----------
# Filenames
vel_x_fn = '../DShean_20171025/Ngoz_tiffs/ngozumpa_20112016_vx_clippedV2.tif'
vel_y_fn = '../DShean_20171025/Ngoz_tiffs/ngozumpa_20112016_vy_clippedV2.tif'
vel_x_std_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_dshean_vx_std_clipV2.tif'
vel_y_std_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_dshean_vy_std_clipV2.tif'
icethickness_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_IceThickness_Huss.tif'
bands_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_Bands_600m_ClippedV2.tif'
# Additional input
mc_simulations = 1000
pixel_size = 30
min_velocity_mean = 5.218
min_velocity_std = 4.352
max_velocity = 200
icethickness_uncertainty_percentage = -21
vel_depth_avg_factor_min = 0.8
vel_depth_avg_factor_max = 0.8
stagnant_band = 11

# ----- Quality Control Options ----------
# Filter min/max velocity
option_minvelocity = 1
option_maxvelocity = 1
#  > 1 (default) - apply limits specified in model input
#  > 0 - do not apply limits
option_border = 1
#  > 1 (default) - apply the border to prevent errors
#  > 0 - do not apply border, assumes errors near edges are avoided elsewhere
option_stagnantbands = 1
#  > 1 (default) - remove velocities from stagnant bands that are caused by cliffs and ponds
#  > 0 - do not remove the clusters of higher velocities on the tongues of the glaciers

In [4]:
# Open raster for properties (will be used to export tif)
ds = gdal.Open(vel_x_fn)
# Import rasters
vel_x = import_raster(vel_x_fn)
vel_y = import_raster(vel_y_fn)
vel_x_std = import_raster(vel_x_std_fn)
vel_y_std = import_raster(vel_y_std_fn)
icethickness = import_raster(icethickness_fn)
bands = import_raster(bands_fn)

In [5]:
# MONTE CARLO SIMULATIONS OF EMERGENCE VELOCITY
# Compute the total velocity
vel_total = (vel_x**2 + vel_y**2)**0.5
#   The total velocity correction, i.e., the minimum velocity based on the total velocity, must be applied prior to the 
#   velocity being adjusted for uncertainties because those uncertainties would change the minimum velocity threshold
#   to the exact same extent.  Hence, the total velocity is computed outside the MC simulations here.
# Generate normal random numbers (mean 0, stdev 1) to include vx and vy uncertainties
mc_vm_uncertainty = np.random.normal(0, 1, (1, mc_simulations))
# Generate normal distribution for minimum velocity threshold
mc_vmin_uncertainty = np.random.normal(min_velocity_mean, min_velocity_std, (1, mc_simulations))
# Generate uniform distribution for ice thickness uncertainty between Huss and -21% underestimation
mc_icethickness_uncertainty = np.random.uniform(icethickness_uncertainty_percentage, 0, (1, mc_simulations))
# Generate uniform distribution for ratio of surface to mean velocity
mc_vsfc_ratio_uncertainty = np.random.uniform(vel_depth_avg_factor_min, vel_depth_avg_factor_max, (1, mc_simulations))
# Run the Monte Carlo simulations
for sim in range(mc_simulations):  
    print('Simulation:', sim)
    # Compute velocities at each pixel according to uncertainty
    mc_vel_x = vel_x + (mc_vm_uncertainty[0, sim] * vel_x_std)
    mc_vel_y = vel_y + (mc_vm_uncertainty[0, sim] * vel_y_std)
    # Minimum velocity including uncertainty
    mc_vmin = mc_vmin_uncertainty[0, sim]
    # Compute ice thicknesses based on uncertainty
    mc_icethickness = icethickness / (1 + mc_icethickness_uncertainty[0, sim]/100)
    #  percent_uncertainty = (x2 - x1) / x1 * 100
    #  percent_uncertainty is the uncertainty (mean = -21%, stdev = 60%)
    #  x2 is the modeled ice thickness (Huss and Farinotti, 2012)
    #  We want to solve for x1, i.e., the ice thickness with the error included
    #    x1 = x2 / (1 + percent_uncertainty / 100)
    mc_vsfc_ratio = mc_vsfc_ratio_uncertainty[0, sim]
    # Emergence Velocity Calculations for each pixel
    emergence_velocity = emergence_pixels(mc_vel_x, mc_vel_y, vel_total, mc_vmin, mc_vsfc_ratio, mc_icethickness, bands)
    # Emergence Velocity Calculations for each band (average the pixel values)
    bands_emergence = band_average(emergence_velocity, bands, icethickness)
    # Record the results
    if sim == 0:
        mc_bands_emergence = bands_emergence
    else:
        mc_bands_emergence = np.concatenate((mc_bands_emergence, bands_emergence), axis=1)
# Calculate statistics of Monte Carlo simulatuions
mc_bands_emergence_wstats = np.zeros((mc_bands_emergence.shape[0],mc_bands_emergence.shape[1]+6))
# Column 01 = band number
bands_col = np.arange(bands.max()+1)
mc_bands_emergence_wstats[:,0] = bands_col
# Column 02 - 06 = median, mean, stdev, min, max
mc_bands_emergence_wstats[:,1] = np.median(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,2] = np.mean(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,3] = np.std(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,4] = np.min(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,5] = np.max(mc_bands_emergence, axis=1)
# Columns 7 - ... = MC simulations
mc_bands_emergence_wstats[:,6:mc_bands_emergence.shape[1]+7] = mc_bands_emergence

Simulation: 0
Mass is conserved? True
-531.435106277
Simulation: 1


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  


Mass is conserved? False
91.5208106041
Simulation: 2
Mass is conserved? True
-32.4250860214
Simulation: 3
Mass is conserved? True
-852.815994263
Simulation: 4
Mass is conserved? True
-521.502146721
Simulation: 5
Mass is conserved? False
251.263640881
Simulation: 6
Mass is conserved? True
-144.904880524
Simulation: 7
Mass is conserved? False
561.440395355
Simulation: 8
Mass is conserved? False
510.512613297
Simulation: 9
Mass is conserved? True
-600.882717133
Simulation: 10
Mass is conserved? True
88.5265464783
Simulation: 11
Mass is conserved? True
87.1990675926
Simulation: 12
Mass is conserved? True
-154.833473206
Simulation: 13
Mass is conserved? True
-257.252084732
Simulation: 14
Mass is conserved? True
-129.60058403
Simulation: 15
Mass is conserved? False
228.589706421
Simulation: 16
Mass is conserved? True
-716.480316162
Simulation: 17
Mass is conserved? False
987.338933945
Simulation: 18
Mass is conserved? True
-528.023138046
Simulation: 19
Mass is conserved? True
14.9516553879
S

Mass is conserved? False
480.091198921
Simulation: 154
Mass is conserved? True
-369.630256653
Simulation: 155
Mass is conserved? False
727.878429413
Simulation: 156
Mass is conserved? True
-409.783259392
Simulation: 157
Mass is conserved? True
-542.325908184
Simulation: 158
Mass is conserved? False
123.660560608
Simulation: 159
Mass is conserved? True
-110.885717392
Simulation: 160
Mass is conserved? False
207.081650734
Simulation: 161
Mass is conserved? False
137.666292191
Simulation: 162
Mass is conserved? False
448.758831024
Simulation: 163
Mass is conserved? False
302.290419579
Simulation: 164
Mass is conserved? False
674.828540802
Simulation: 165
Mass is conserved? True
-12.4081306458
Simulation: 166
Mass is conserved? True
-65.3892688751
Simulation: 167
Mass is conserved? False
269.842851639
Simulation: 168
Mass is conserved? True
-133.022911072
Simulation: 169
Mass is conserved? True
-232.91308403
Simulation: 170
Mass is conserved? True
-86.0965156555
Simulation: 171
Mass is con

Mass is conserved? False
286.545024872
Simulation: 304
Mass is conserved? False
499.310585022
Simulation: 305
Mass is conserved? False
181.909154892
Simulation: 306
Mass is conserved? False
131.760477066
Simulation: 307
Mass is conserved? True
-534.471515656
Simulation: 308
Mass is conserved? True
-125.679876328
Simulation: 309
Mass is conserved? False
138.908842087
Simulation: 310
Mass is conserved? False
337.702022552
Simulation: 311
Mass is conserved? False
173.321922302
Simulation: 312
Mass is conserved? True
-377.367420197
Simulation: 313
Mass is conserved? True
-504.547881126
Simulation: 314
Mass is conserved? False
356.376607895
Simulation: 315
Mass is conserved? False
509.363044262
Simulation: 316
Mass is conserved? True
-993.631938934
Simulation: 317
Mass is conserved? True
-591.38441658
Simulation: 318
Mass is conserved? False
518.934456825
Simulation: 319
Mass is conserved? True
-155.574203491
Simulation: 320
Mass is conserved? True
-387.082660675
Simulation: 321
Mass is con

Mass is conserved? False
188.164421082
Simulation: 454
Mass is conserved? True
-36.8195209503
Simulation: 455
Mass is conserved? False
109.228644848
Simulation: 456
Mass is conserved? True
23.6125097275
Simulation: 457
Mass is conserved? False
829.070814133
Simulation: 458
Mass is conserved? False
316.229772568
Simulation: 459
Mass is conserved? False
114.277050018
Simulation: 460
Mass is conserved? True
31.1826877594
Simulation: 461
Mass is conserved? True
-41.6034555435
Simulation: 462
Mass is conserved? False
90.4626941681
Simulation: 463
Mass is conserved? False
235.461670876
Simulation: 464
Mass is conserved? True
-420.767817497
Simulation: 465
Mass is conserved? False
150.001335144
Simulation: 466
Mass is conserved? False
244.302416801
Simulation: 467
Mass is conserved? True
-243.486886024
Simulation: 468
Mass is conserved? True
-458.984957695
Simulation: 469
Mass is conserved? True
-462.521459579
Simulation: 470
Mass is conserved? False
565.452257156
Simulation: 471
Mass is cons

Mass is conserved? False
370.977237701
Simulation: 604
Mass is conserved? True
-270.603471756
Simulation: 605
Mass is conserved? False
535.777814865
Simulation: 606
Mass is conserved? False
231.378526688
Simulation: 607
Mass is conserved? False
1067.14503479
Simulation: 608
Mass is conserved? True
-99.9836330414
Simulation: 609
Mass is conserved? False
439.835142136
Simulation: 610
Mass is conserved? True
-176.142514229
Simulation: 611
Mass is conserved? False
752.586057663
Simulation: 612
Mass is conserved? False
340.628072739
Simulation: 613
Mass is conserved? False
351.044427872
Simulation: 614
Mass is conserved? False
374.528484344
Simulation: 615
Mass is conserved? True
-462.552189827
Simulation: 616
Mass is conserved? True
-127.268666267
Simulation: 617
Mass is conserved? True
40.5060853958
Simulation: 618
Mass is conserved? True
25.0944576263
Simulation: 619
Mass is conserved? False
258.638567924
Simulation: 620
Mass is conserved? False
309.526370049
Simulation: 621
Mass is cons

Mass is conserved? True
-289.48236084
Simulation: 754
Mass is conserved? False
278.674066544
Simulation: 755
Mass is conserved? False
308.882073402
Simulation: 756
Mass is conserved? False
280.473088264
Simulation: 757
Mass is conserved? True
-193.732273102
Simulation: 758
Mass is conserved? False
367.135167122
Simulation: 759
Mass is conserved? True
-322.912683487
Simulation: 760
Mass is conserved? False
166.422353745
Simulation: 761
Mass is conserved? True
-659.002737045
Simulation: 762
Mass is conserved? False
422.724869728
Simulation: 763
Mass is conserved? True
-180.502984047
Simulation: 764
Mass is conserved? False
997.533694267
Simulation: 765
Mass is conserved? True
-89.8522911072
Simulation: 766
Mass is conserved? False
285.850735664
Simulation: 767
Mass is conserved? False
718.896142006
Simulation: 768
Mass is conserved? False
409.088995934
Simulation: 769
Mass is conserved? True
-166.97752285
Simulation: 770
Mass is conserved? True
-193.335213661
Simulation: 771
Mass is cons

Mass is conserved? False
742.819180489
Simulation: 904
Mass is conserved? False
1067.47014046
Simulation: 905
Mass is conserved? True
60.2040128708
Simulation: 906
Mass is conserved? False
389.72750473
Simulation: 907
Mass is conserved? True
74.3110103607
Simulation: 908
Mass is conserved? False
151.460594177
Simulation: 909
Mass is conserved? True
-30.6809482574
Simulation: 910
Mass is conserved? True
-513.548042297
Simulation: 911
Mass is conserved? False
471.733829498
Simulation: 912
Mass is conserved? False
691.689487457
Simulation: 913
Mass is conserved? False
495.709579468
Simulation: 914
Mass is conserved? True
-754.037679672
Simulation: 915
Mass is conserved? False
495.405468941
Simulation: 916
Mass is conserved? False
250.141983032
Simulation: 917
Mass is conserved? False
126.979001999
Simulation: 918
Mass is conserved? False
359.918692589
Simulation: 919
Mass is conserved? True
12.1062498093
Simulation: 920
Mass is conserved? False
630.739564896
Simulation: 921
Mass is conser

C:\Users\David\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3858: RuntimeWarning: Invalid value encountered in median for 1 results
  r = func(a, **kwargs)


In [6]:
np.savetxt('../Output/emergence_Ngozumpa_mcstat_stagnantbands.txt', mc_bands_emergence_wstats)
# np.savetxt('../Output/emergence_Khumbu_mcstat_stagnantbands.txt', mc_bands_emergence_wstats)
# np.savetxt('../Output/emergence_Khumbu.txt', bands_emergence)

In [7]:
# # Run without uncertainties
# vel_total = (vel_x**2 + vel_y**2)**0.5
# emergence_velocity = emergence_pixels(vel_x, vel_y, vel_total, icethickness)
# bands_emergence = band_average(emergence_velocity, bands, icethickness)
# print(bands_emergence)

In [8]:
# plt.figure(1)
# plt.imshow(vel_x)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('x velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(2)
# plt.imshow(vel_y)
# plt.title('y velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(3)
# plt.imshow(icethickness)
# plt.title('ice thickness (m)')
# plt.colorbar()

# vel_total = total_velocity(vel_x, vel_y)
# plt.figure(4)
# plt.imshow(vel_total)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('total velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()